In [1]:
import numpy as np
from langmodels.models import *
import langmodels.utf8codec as utf8codec
from langmodels.utils.tools import *
import torch.nn.functional as F
import torch.nn as nn
import torch

Loading faiss with AVX2 support.
Loading faiss.


In [2]:
from fairseq.modules.transformer_layer import MultiheadAttention, TransformerDecoderLayer, TransformerEncoderLayer

In [3]:
in_conv_dim=1024
conv_proj_dim=256
att_layers=2
att_dim=128
att_encoder_heads=8
att_encoder_ff_embed_dim=1024
dropout=0.1
att_dropout=0.1
activation=None
residual=True

args = SimpleNamespace(**{"encoder_embed_dim": att_dim, "decoder_embed_dim": att_dim,
                                  "encoder_attention_heads": att_encoder_heads, "decoder_attention_heads": att_encoder_heads,
                                  "attention_dropout": att_dropout,
                                  "dropout": dropout,
                                  "activation_fn": "gelu",
                                  "encoder_normalize_before": True, "decoder_normalize_before": True,
                                  "encoder_ffn_embed_dim": att_encoder_ff_embed_dim, "decoder_ffn_embed_dim": att_encoder_ff_embed_dim,
                                  })

In [4]:
args

namespace(activation_fn='gelu', attention_dropout=0.1, decoder_attention_heads=8, decoder_embed_dim=128, decoder_ffn_embed_dim=1024, decoder_normalize_before=True, dropout=0.1, encoder_attention_heads=8, encoder_embed_dim=128, encoder_ffn_embed_dim=1024, encoder_normalize_before=True)

In [5]:
tenc = TransformerDecoderLayer(args)

In [6]:
conv_col = Conv1DColNet()

In [7]:
count_parameters(conv_col), count_parameters(conv_col)

(2173824, 2173824)

In [8]:
conv_col

Conv1DColNet(
  (_input_encoder): Sequential(
    (0): Linear(in_features=324, out_features=768, bias=True)
    (1): Linear(in_features=768, out_features=96, bias=True)
  )
  (_encoder): Conv1DEncoder(
    (_blocks): ModuleList(
      (0): Conv1DBlock(
        (convresid): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
        (convs): Sequential(
          (0): Conv1d(96, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          (5): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): Conv1DBlock(
        (convresid): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
        (convs): Sequential(


In [9]:
convatt_col = ConvAttColNet(conv_col)

In [10]:
count_parameters(convatt_col), count_parameters(convatt_col)

(12838560, 12838560)

In [11]:
convatt_col

ConvAttColNet(
  (_convcolnet): Conv1DColNet(
    (_input_encoder): Sequential(
      (0): Linear(in_features=324, out_features=768, bias=True)
      (1): Linear(in_features=768, out_features=96, bias=True)
    )
    (_encoder): Conv1DEncoder(
      (_blocks): ModuleList(
        (0): Conv1DBlock(
          (convresid): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
          (convs): Sequential(
            (0): Conv1d(96, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (5): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): Conv1DBlock(
          (convresid): Conv1

In [25]:
convatt_col = ConvAttColNet(conv_col, 
                 in_dim=324, hidd_embed_dim=768, out_embed_dim=96,  # input Embedding adaptor channel-wise
                 in_conv_channels=96, lin_channels=96,
                 in_conv_dim=1024, conv_proj_dim=256, att_layers=2, att_dim=256,
                 att_encoder_heads=16, att_encoder_ff_embed_dim=1024,
                 dropout=0.1, att_dropout=0.1, activation=None, residual=True,
                 out_in_dim=96, out_hidd_embed_dim=1024,  # decoder channel_wise dimensions
                 out_seq_len=512,  # output sequence length, maximum attention that will appear there
                           )

In [26]:
count_parameters(convatt_col), count_parameters(convatt_col)

(18025888, 18025888)

In [14]:
convatt_col

ConvAttColNet(
  (_convcolnet): Conv1DColNet(
    (_input_encoder): Sequential(
      (0): Linear(in_features=324, out_features=768, bias=True)
      (1): Linear(in_features=768, out_features=96, bias=True)
    )
    (_encoder): Conv1DEncoder(
      (_blocks): ModuleList(
        (0): Conv1DBlock(
          (convresid): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
          (convs): Sequential(
            (0): Conv1d(96, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (5): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): Conv1DBlock(
          (convresid): Conv1

In [28]:
convatt_col.to("cuda:0")

ConvAttColNet(
  (_convcolnet): Conv1DColNet(
    (_input_encoder): Sequential(
      (0): Linear(in_features=324, out_features=768, bias=True)
      (1): Linear(in_features=768, out_features=96, bias=True)
    )
    (_encoder): Conv1DEncoder(
      (_blocks): ModuleList(
        (0): Conv1DBlock(
          (convresid): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
          (convs): Sequential(
            (0): Conv1d(96, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (5): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): Conv1DBlock(
          (convresid): Conv1

This convatt_col module is aprox 600-700MB in the GPU

In [15]:
# tcn_col = TCNColumn(1024, [128])
tcn_col = TCNColumn()

In [16]:
count_parameters(tcn_col)

412352

In [17]:
256+128

384

In [18]:
utf8codes = np.load("./utf8-codes/utf8_codebook_overfit_matrix_2seg_dim64.npy")
utf8codes = utf8codes.reshape(1987,64)

In [19]:
gconv_net = GatedConv1DPoS(utf8codes)

In [20]:
count_parameters(gconv_net), count_trainable_parameters(gconv_net)

(3688304, 3622768)